In [1]:
import pandas as pd
from scipy import stats
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
movies_df = pd.read_csv('movies.csv')
ratings_df= pd.read_csv('ratings.csv')

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))','')
movies_df['title'] = movies_df['title'].apply(lambda x:x.strip())

In [5]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji,Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men,Comedy|Romance,(1995)
3,4,Waiting to Exhale,Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II,Comedy,(1995)


In [6]:
movies_df = movies_df.drop('genres',1)

In [7]:
#ratings_df_x= ratings_df.groupby(['timestamp'])
ratings_df_x = ratings_df.sort_values(by='timestamp',ascending= True)
#ratings_df_x.head()

In [8]:
users_df= ratings_df.groupby(['movieId'])
#users_df.get_group(1176)

In [9]:
ratings_df.head(1000)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [10]:
userInput = [
            {'movieId':2471, 'rating':4, 'timestamp':1112484203},
            {'movieId':2, 'rating':2,  'timestamp':1102784203},
            {'movieId':158, 'rating':4, 'timestamp':1112482303},
            {'movieId':1274, 'rating':4.5, 'timestamp':1122484203},
            {'movieId':170, 'rating':5, 'timestamp':1114484203}
        ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,movieId,rating,timestamp
0,2471,4.0,1112484203
1,2,2.0,1102784203
2,158,4.0,1112482303
3,1274,4.5,1122484203
4,170,5.0,1114484203


In [11]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
451,5,2,3.0,851527569
970,11,158,4.0,1230853973
973,11,170,5.0,1230782534
1057,11,1274,5.0,1230784185


In [12]:
usersubgroup = userSubset.groupby(['userId'])
usersubgroup = sorted(usersubgroup, key=lambda x:len(x[1]),reverse=True)
#usersubgroup[0:100]

In [15]:
cordict = {}

for name,group in usersubgroup:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by = 'movieId')
    
    nRatings = len(group)
    
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    temp_rating_list_rating = temp_df['rating'].tolist()
    temp_group_list_rating = group['rating'].tolist()
    temp_rating_list_timestamp = temp_df['timestamp'].tolist()
    temp_group_list_timestamp = group['timestamp'].tolist()
    
    
    Rxx = sum([i**2 for i in temp_rating_list_rating]) - pow(sum(temp_rating_list_rating),2)/float(nRatings)
    Ryy = sum([i**2 for i in temp_group_list_rating]) - pow(sum(temp_group_list_rating),2)/float(nRatings)
    Rxy = sum(i*j for i,j in zip(temp_rating_list_rating,temp_group_list_rating)) - sum(temp_rating_list_rating)*sum(temp_group_list_rating)/float(nRatings)
    
    Txx = sum([i**2 for i in temp_rating_list_timestamp]) - pow(sum(temp_rating_list_timestamp),2)/float(nRatings)
    Tyy = sum([i**2 for i in temp_group_list_timestamp]) - pow(sum(temp_group_list_timestamp),2)/float(nRatings)
    Txy = sum(i*j for i,j in zip(temp_rating_list_timestamp,temp_group_list_timestamp)) - sum(temp_rating_list_timestamp)*sum(temp_group_list_timestamp)/float(nRatings)
    
    if Txx!=0 and Tyy!=0 and Rxx!=0 and Ryy!=0:
        cordict[name] = Rxy/sqrt(Rxx*Ryy) + (Txy/sqrt(Txx*Tyy))/2
    else:
        cordict[name]=0

In [16]:
cordict.items()

dict_items([(1, 0), (135300, 0), (65539, -0.5000132392157288), (131077, 1.4994376862278116), (65542, 0), (109985, 0), (11, 0.5463570168510941), (13, 0), (131086, 0), (131088, -0.3386524805791041), (131092, 0), (21, 0), (65562, 0), (109231, 0), (131100, 0), (29, 0), (131102, 0), (5, 0), (65568, 0), (65569, 0), (34, 0), (65573, 0), (134060, 0), (131113, -0.5001468643508197), (65582, 0), (51, 0), (54, 0), (126134, 0), (65592, -0.5014460155120681), (58, 0), (54623, 1.4971844145105881), (61, 0), (62, 0), (131138, -0.4998959876861886), (67, 0), (68, 0), (65605, 0), (65607, 0), (65608, 0), (131145, 0), (65610, 0), (43703, 0), (76, 0), (77, 1.5003411141382959), (78, 0), (65618, 0), (131155, 0), (86, 0), (88, 0), (131161, 0), (93840, 0), (91, -1.5000000036767276), (65628, 0), (65630, -0.4986867734342839), (132681, 0), (109243, 0), (131172, 0), (131173, 0), (65638, 0), (65639, 0), (104, 0), (131177, 0), (134496, 0), (131180, 0.9059368576181979), (109, 0), (109245, 0.8604884424519421), (91567, 0)

In [17]:
pearsonDF = pd.DataFrame.from_dict( cordict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.000000,1
1,0.000000,135300
2,-0.500013,65539
3,1.499438,131077
4,0.000000,65542


In [18]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
23393,1.662913,39374
28996,1.596622,114747
12710,1.562500,20657
7432,1.562499,77073
29254,1.552427,115224


In [19]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating,timestamp
0,1.662913,39374,1,4.0,835542750
1,1.662913,39374,2,3.0,835542862
2,1.662913,39374,5,5.0,835543030
3,1.662913,39374,19,3.0,835542786
4,1.662913,39374,27,5.0,835543046


In [20]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,timestamp,weightedRating
0,1.662913,39374,1,4.0,835542750,6.651650
1,1.662913,39374,2,3.0,835542862,4.988738
2,1.662913,39374,5,5.0,835543030,8.314563
3,1.662913,39374,19,3.0,835542786,4.988738
4,1.662913,39374,27,5.0,835543046,8.314563


In [21]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,29.181691,109.882465
2,71.892802,200.609780
3,10.676438,44.297363
4,6.092957,16.697329
5,15.527833,57.503537


In [22]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.765459,1
2,2.790401,2
3,4.149077,3
4,2.740431,4
5,3.703256,5


In [23]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
88744,5.0,88744
1944,5.0,1944
4896,5.0,4896
312,5.0,312
4826,5.0,4826
1735,5.0,1735
1753,5.0,1753
4612,5.0,4612
335,5.0,335


In [24]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
309,312,Stuart Saves His Family,(1995)
331,335,Underneath,(1995)
1675,1735,Great Expectations,(1998)
1688,1753,Half Baked,(1998)
1799,1883,Bulworth,(1998)
1860,1944,From Here to Eternity,(1953)
4517,4612,Jesus of Montreal (Jésus de Montréal),(1989)
4730,4826,"Big Red One, The",(1980)
4800,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,(2001)
17650,88744,Rise of the Planet of the Apes,(2011)
